# proboscidean fossil map dash app


The goal of this project is to build up on the previous plotly project where we grabbed a table containing a list of [Proboscidean](https://fr.wikipedia.org/wiki/Proboscidea) (elephants and related extinct species) fossils from [paleobiodb.org](paleobiodb.org) and built a map of these fossils.

The idea will be to add the possibility to add some way to select a point in time, or a range, and to represent only the fossils which are compatible with the selected time.


In [13]:
## reading the data
import pandas as pd

df = pd.read_csv("data/proboscidean_fossils.pbdb.csv")
df.head()

,occurrence_no,record_type,reid_no,flags,collection_no,identified_name,identified_rank,identified_no,difference,accepted_name,...,paleomodel2,geoplate2,paleoage2,paleolng2,paleolat2,paleomodel3,geoplate3,paleoage3,paleolng3,paleolat3
0,138745,occ,NaN,NaN,11798,Serridentinus ? sp.,genus,51518,subjective synonym of,Gomphotherium,...,scotese,611,mid,101.03,24.39,seton,602,mid,100.92,26.02
1,138746,occ,NaN,NaN,11798,Tetralophodon cf. exoletus,species,43261,species not entered,Tetralophodon,...,scotese,611,mid,101.03,24.39,seton,602,mid,100.92,26.02
2,138747,occ,NaN,NaN,11798,Zygolophodon n. sp. lufengensis,species,43277,species not entered,Zygolophodon,...,scotese,611,mid,101.03,24.39,seton,602,mid,100.92,26.02
3,147930,occ,NaN,NaN,13060,Stegodon bombifrons,species,476344,NaN,Stegodon bombifrons,...,scotese,501,mid,77.67,28.48,seton,501,mid,74.98,29.09
4,150057,occ,NaN,NaN,13293,Elephas cf. kiangnanensis,species,43264,species not entered,Elephas,...,scotese,not computable using this model,mid,NaN,NaN,seton,not computable using this model,mid,NaN,NaN


In [33]:
## fossil age range
df[['max_ma', 'min_ma']].head()

,max_ma,min_ma
0,11.620,7.2460
1,11.620,7.2460
2,11.620,7.2460
3,11.608,5.3330
4,0.781,0.0117


We also use modern shorelines from https://www.naturalearthdata.com/ :

In [14]:
coastlines = pd.read_csv("data/coastines.csv")
coastlines.head()

,lon,lat
0,-163.712896,-78.595667
1,-163.105801,-78.223339
2,-161.245113,-78.380177
3,-160.246208,-78.693646
4,-159.482405,-79.046338


And we got this visualization from the plotly project. 

You can use this or you can use what you had created yourself.

In [44]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np


# Create figure with subplots
fig = make_subplots(rows=1, cols=2)


### subplot 1

# coastlines
fig.add_trace( go.Scatter( visible=True, 
                          x=coastlines.lon, y=coastlines.lat,
                          mode='lines',
                          showlegend=False,
                          hoverinfo="none" 
                         ) ,
              row=1, col=1  # this is how we say in which subplot this traces goes
             )
def make_label(row):
    return f"{row.accepted_name} ({row.identified_rank})<br>" + f"{row.max_ma:.1f}MYA - {row.min_ma:.1f}MYA"
    

# add fossils
fig.add_trace( go.Scatter( x=df['lng'],
                           y=df['lat'], 
                           opacity=0.75 , 
                           mode='markers',
                           hovertemplate = "%{text}<br>"+ # custom label
                                         "coords : (%{y:.1f}N, %{x:.1f}E)" ,
                          text = [make_label(row) for i,row in  df.iterrows()],
                          showlegend=False 
                         ),
              row=1, col=1
             )


### subplot 2
R = np.arange(-60, 0.1, 1) ## define a timeline, from -60MYA to now
nG = [df.loc[ (df.max_ma > -T) & (df.min_ma < -T) , 'genus' ].nunique() for T in R] ## count number of genus at each time

fig.add_trace( go.Scatter( x=R, 
                           y=nG, 
                           mode='lines',showlegend=False),
                           row=1, col=2) 

fig

**TASK:**

 1. add a slider corresponding to time to show only the fossils that could be present at that time (ie, their min_ma and max_ma are compatible with the shown time)
 2. visuallize the chosen time on the second plot
 
 Useful link: [dash core component](https://dash.plotly.com/dash-core-components)

Solution:

In [66]:
# %load solutions/solution_dash_proboscideans.py